In [1]:
!pip install pyspark==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=cefd19336b2aae60c56866b427f51acbf143762340daa282d2198d29f7df57bc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
# Conectando ao google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.appName("onehot").getOrCreate()

In [7]:
from pyspark.ml.feature import OneHotEncoder

In [8]:
churn = spark.read.load("/content/drive/MyDrive/Colab Notebooks/Spark/datas/Churn.csv", format="csv", sep=";", inferSchema=True, header=True)
churn.show(5)

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-------

In [13]:
# Usando string index para transformar coluna categorica para numerica - Coluna Geography e Gender


# Criando os indices
from pyspark.ml.feature import StringIndexer

indice = StringIndexer(inputCol="Geography", outputCol="Indexer_C1")
indiceonehot = indice.fit(churn).transform(churn)

indice = StringIndexer(inputCol="Gender", outputCol="Indexer_C2")
indiceonehot = indice.fit(churn).transform(indiceonehot)

indiceonehot.select("Indexer_C1", "Indexer_C2").show()


+----------+----------+
|Indexer_C1|Indexer_C2|
+----------+----------+
|       0.0|       1.0|
|       2.0|       1.0|
|       0.0|       1.0|
|       0.0|       1.0|
|       2.0|       1.0|
|       2.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       2.0|       0.0|
|       0.0|       1.0|
|       0.0|       1.0|
|       2.0|       1.0|
|       1.0|       0.0|
|       1.0|       0.0|
|       2.0|       1.0|
|       2.0|       0.0|
|       0.0|       1.0|
+----------+----------+
only showing top 20 rows



In [15]:
onehot = OneHotEncoder(inputCols = ["Indexer_C1", "Indexer_C2"], outputCols=["OneHot_C1", "OneHot_C2"])
modelo = onehot.fit(indiceonehot)
onehotout = modelo.transform(indiceonehot)
onehotout.select("OneHot_C1", "OneHot_C2").show(truncate=False)

+-------------+-------------+
|OneHot_C1    |OneHot_C2    |
+-------------+-------------+
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[1],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[1],[1.0])|(1,[0],[1.0])|
|(2,[1],[1.0])|(1,[0],[1.0])|
|(2,[],[])    |(1,[],[])    |
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[],[])    |
+-------------+-------------+
only showing top 20 rows

